In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pprint import pprint

%matplotlib notebook

In [35]:
#load trending videos
trending_videos = pd.read_csv('USvideos.csv')
top_trending_videos_days = trending_videos.groupby(['title']).size().reset_index(name='trending_days_count').sort_values(['trending_days_count'], ascending=[False])
#top_trending_videos_days['trending_days_count'].head(20).sum()
top_trending_videos_days

,title,trending_days_count
3618,"Selena Gomez, Marshmello - Wolves",18
2760,Maroon 5 - Wait,18
2622,Lucas the Spider - Polar Bear,16
849,Chris Young - Hangin' On,16
51,*cough*,16
1425,Fluffy Sleepy Whispers ASMR,15
4705,Why Bridges Move...,15
1571,Gotta Catch ‘Em All!,15
2691,Made in Miami (Artist Spotlight Story) - Camil...,15
3364,Ralph Breaks The Internet: Wreck-It Ralph 2 Of...,15


In [41]:
#update types
type_int_list = ['views', 'likes', 'dislikes', 'comment_count']
for column in type_int_list:
    trending_videos[column] = trending_videos[column].astype(int)

type_str_list = ['category_id']
for column in type_str_list:
    trending_videos[column] = trending_videos[column].astype(str)

In [42]:
# load id category
# creates a dictionary that maps `category_id` to `category`
id_to_category = {}

with open('US_category_id.json', 'r') as f:
    data = json.load(f)
    for category in data['items']:
        id_to_category[category['id']] = category['snippet']['title']

trending_videos['category'] = trending_videos['category_id'].map(id_to_category)


In [43]:
trending_videos.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category
0,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,People & Blogs
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",Entertainment
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,Comedy
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,Entertainment
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,Entertainment


In [60]:
def plot_youtube(y_axis,num_trending_videos, cat):
    plt.figure(figsize=(10, 10))
    if cat == 'ALL':
        sns.barplot(x='title', y=y_axis, data=trending_videos.sort_values([y_axis], ascending=[False]).head(num_trending_videos))
    else:
        sns.barplot(x='title', y=y_axis, data=trending_videos[trending_videos['category'] == cat].sort_values([y_axis], ascending=[False]).head(num_trending_videos))
    
    plt.title('Top Videos by ' + y_axis.capitalize())
    plt.xticks(rotation=90)
    sns.set(style="darkgrid")
    plt.gcf().subplots_adjust(bottom=.6)
    

In [69]:
#plot_youtube('views',100,'ALL')
#plot_youtube('likes',100,'ALL')
#plot_youtube('comment_count',100,'ALL')
trending_videos[trending_videos['category'] == 'Entertainment']   

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category
1,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John...",Entertainment
3,puqaWrEC7tY,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...,Entertainment
4,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,Entertainment
6,39idVpFF7NQ,17.14.11,Roy Moore & Jeff Sessions Cold Open - SNL,Saturday Night Live,24,2017-11-12T05:37:17.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",2103417,15993,2445,1970,https://i.ytimg.com/vi/39idVpFF7NQ/default.jpg,False,False,False,Embattled Alabama Senate candidate Roy Moore (...,Entertainment
11,VifQlJit6A0,17.14.11,(SPOILERS) 'Shiva Saves the Day' Talked About ...,amc,24,2017-11-13T03:00:00.000Z,"The Walking Dead|""shiva""|""tiger""|""king ezekiel...",104578,1576,303,1279,https://i.ytimg.com/vi/VifQlJit6A0/default.jpg,False,False,False,Shiva arrives just in time as King Ezekiel att...,Entertainment
25,dRpNZV18N_g,17.14.11,SPAGHETTI BURRITO VS SPAGHETTI BURRITO,HellthyJunkFood,24,2017-11-12T14:00:04.000Z,"spaghetti burrito|""diy burrito""|""spaghetti""|""b...",223077,8676,193,1392,https://i.ytimg.com/vi/dRpNZV18N_g/default.jpg,False,False,False,Visit http://www.Bongiovibrand.com\nand get 20...,Entertainment
31,CBVGjS_EJok,17.14.11,Amazon Christmas Advert 2017 - Toys & Games,Amazon.co.uk,24,2017-11-06T17:52:50.000Z,"Amazon|""Amazon Christmas""|""Amazon Xmas""|""Chris...",26000,119,69,0,https://i.ytimg.com/vi/CBVGjS_EJok/default.jpg,True,False,False,Shhhh. See how Amazon helps Dad create a magic...,Entertainment
33,hz7ukDjuq4w,17.14.11,What's Inside a Detectives Car?,officer401,24,2017-11-12T23:41:37.000Z,"detective|""officer""|""401""|""officer401""|""police...",67661,3781,84,626,https://i.ytimg.com/vi/hz7ukDjuq4w/default.jpg,False,False,False,Intro Song: Dion Timmer - Lost\nEnding Song: S...,Entertainment
46,ogYum4kWXgk,17.14.11,People are Awesome & The Pet Collective presen...,People are Awesome,24,2017-11-13T13:00:06.000Z,"people are awesome|""people are awesome 2017""|""...",69844,3417,33,160,https://i.ytimg.com/vi/ogYum4kWXgk/default.jpg,False,False,False,Pets are Awesome! This is for all the animal l...,Entertainment
49,DM-ni_LSOFE,17.14.11,"ELDERS REACT TO iPHONE X (Facial Recognition, ...",FBE,24,2017-11-10T23:05:25.000Z,"iPhone X|""animojis""|""facial recognition""|""ELDE...",2045386,45406,2842,8309,https://i.ytimg.com/vi/DM-ni_LSOFE/default.jpg,False,False,False,"iPhoneX (Facial Recognition, Animojis) reacted...",Entertainment


In [ ]:
### Question 1:  Choose one variable and plot that variable four different ways.
#top 15 counts
#plot 1
plt.figure(figsize=(10, 5))
sns.barplot(x="title", y="views", data=trending_videos)
plt.title('Plot 1:  Barplot - Top 15 Police Fatality Count Cities in the United States annually based on data from 2015 - 2017')
plt.xticks(rotation=90)
sns.set(style="darkgrid")

g = sns.factorplot(x="gender", y="age", data=police_killings,
                   size=6, kind="bar", palette="pastel", ci=95)
g.despine(left=True)
g.set_ylabels("Age")
g.set_xlabels("")
plt.title('Plot 1A:  Barplot - Age by Gender')
plt.show()



#plot 2
fig, ax = plt.subplots()
fig.set_size_inches(10,5)
plt.hist(df_killings_grouped_city_state_median_population['death_count_per_million_per_year'], color='red',label='Death coiunt')  # alpha just controls the opacity
plt.xlabel('Death count per million per year')
plt.legend(loc='upper right')
plt.title('Plot 2: Histogram - Death count per million per year')
plt.show()

#plot 3
# Setting the overall aesthetic.
fig, ax = plt.subplots()
fig.set_size_inches(10,5)
sns.set(style="whitegrid")
ax = sns.boxplot(x='death_count_per_million_per_year',data=df_killings_grouped_city_state_median_population,palette='pastel')  
plt.title('Plot 3: Boxplot - Death count per million per year')
sns.despine(offset=10, trim=True)
ax.set(xlabel='Death count per million per year')
plt.show()

#plot 4
# Setting the overall aesthetic.
sns.set(style="whitegrid")
g = sns.factorplot(y="death_count_per_million_per_year", data=df_killings_grouped_city_state_median_population,
                   size=6, kind="point", palette="pastel",ci=95,dodge=True,join=False)
#g.despine(left=True)
g.set(xlabel='Death count per million per year')
plt.title('Plot 4:  Pointplot - Death count per million per year')
plt.show()

In [11]:
trending_videos.sort_values(['views'], ascending=[False])

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
6181,FlsCjmMhFmw,17.14.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06T17:58:51.000Z,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",149376127,3093544,1643059,810698,https://i.ytimg.com/vi/FlsCjmMhFmw/default.jpg,False,False,False,"YouTube Rewind 2017. Celebrating the videos, p..."
5935,FlsCjmMhFmw,17.13.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06T17:58:51.000Z,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",137843120,3014471,1602383,817582,https://i.ytimg.com/vi/FlsCjmMhFmw/default.jpg,False,False,False,"YouTube Rewind 2017. Celebrating the videos, p..."
5699,FlsCjmMhFmw,17.12.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06T17:58:51.000Z,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",125432237,2912702,1545015,807558,https://i.ytimg.com/vi/FlsCjmMhFmw/default.jpg,False,False,False,"YouTube Rewind 2017. Celebrating the videos, p..."
5452,FlsCjmMhFmw,17.11.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06T17:58:51.000Z,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",113874632,2811215,1470383,787174,https://i.ytimg.com/vi/FlsCjmMhFmw/default.jpg,False,False,False,"YouTube Rewind 2017. Celebrating the videos, p..."
2587,TyHvyGVs42U,17.26.11,"Luis Fonsi, Demi Lovato - Échame La Culpa",LuisFonsiVEVO,10,2017-11-17T05:00:01.000Z,"Luis|""Fonsi""|""Demi""|""Lovato""|""Échame""|""La""|""Cu...",102012605,2376636,117196,134224,https://i.ytimg.com/vi/TyHvyGVs42U/default.jpg,False,False,False,“Échame La Culpa” disponible ya en todas las p...
5236,FlsCjmMhFmw,17.10.12,YouTube Rewind: The Shape of 2017 | #YouTubeRe...,YouTube Spotlight,24,2017-12-06T17:58:51.000Z,"Rewind|""Rewind 2017""|""youtube rewind 2017""|""#Y...",100911567,2656659,1353647,682890,https://i.ytimg.com/vi/FlsCjmMhFmw/default.jpg,False,False,False,"YouTube Rewind 2017. Celebrating the videos, p..."
4997,6ZfuNTqbHE8,17.08.12,Marvel Studios' Avengers: Infinity War Officia...,Marvel Entertainment,24,2017-11-29T13:26:24.000Z,"marvel|""comics""|""comic books""|""nerdy""|""geeky""|...",91933007,2625661,53709,350458,https://i.ytimg.com/vi/6ZfuNTqbHE8/default.jpg,False,False,False,There was an idea… Avengers: Infinity War. In ...
2365,TyHvyGVs42U,17.25.11,"Luis Fonsi, Demi Lovato - Échame La Culpa",LuisFonsiVEVO,10,2017-11-17T05:00:01.000Z,"Luis|""Fonsi""|""Demi""|""Lovato""|""Échame""|""La""|""Cu...",91552137,2277789,110752,128071,https://i.ytimg.com/vi/TyHvyGVs42U/default.jpg,False,False,False,“Échame La Culpa” disponible ya en todas las p...
4743,6ZfuNTqbHE8,17.07.12,Marvel Studios' Avengers: Infinity War Officia...,Marvel Entertainment,24,2017-11-29T13:26:24.000Z,"marvel|""comics""|""comic books""|""nerdy""|""geeky""|...",89930713,2606662,53011,347982,https://i.ytimg.com/vi/6ZfuNTqbHE8/default.jpg,False,False,False,There was an idea… Avengers: Infinity War. In ...
4514,6ZfuNTqbHE8,17.06.12,Marvel Studios' Avengers: Infinity War Officia...,Marvel Entertainment,24,2017-11-29T13:26:24.000Z,"marvel|""comics""|""comic books""|""nerdy""|""geeky""|...",87449453,2584672,52176,341571,https://i.ytimg.com/vi/6ZfuNTqbHE8/default.jpg,False,False,False,There was an idea… Avengers: Infinity War. In ...
